In [1]:
import pandas as pd
import requests
import dbconfig as dbConf
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
def getLottoWinInfo(startRound, endRound): 
    roundNo = []
    drwtNo1 = [] 
    drwtNo2 = [] 
    drwtNo3 = [] 
    drwtNo4 = [] 
    drwtNo5 = [] 
    drwtNo6 = [] 
    bnusNo = [] 
     
    for i in range(startRound, endRound+1, 1): 
        req_url = "https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo=" + str(i) 
        
        req_lotto = requests.get(req_url) 
        lottoNo = req_lotto.json() 
        roundNo.append(i)
        drwtNo1.append(lottoNo['drwtNo1']) 
        drwtNo2.append(lottoNo['drwtNo2']) 
        drwtNo3.append(lottoNo['drwtNo3']) 
        drwtNo4.append(lottoNo['drwtNo4']) 
        drwtNo5.append(lottoNo['drwtNo5']) 
        drwtNo6.append(lottoNo['drwtNo6']) 
        bnusNo.append(lottoNo['bnusNo']) 
         
        lotto_dict = {"seq":roundNo, 
                      "n1":drwtNo1, 
                      "n2":drwtNo2, 
                      "n3":drwtNo3, 
                      "n4":drwtNo4, 
                      "n5":drwtNo5, 
                      "n6":drwtNo6} 

        lotto_df = pd.DataFrame(lotto_dict) 
        
    return lotto_df 

In [3]:
def insert_new_number(lotto_df):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    
    for i in range(len(lotto_df)):
        first  = "replace into zero.lotto values(" 
        middle = ","
        last   = ");"
        insert_sql = (
                        first 
                      + str(lotto_df['seq'][i]) 
                      + middle
                      + str(lotto_df['n1'][i])
                      + middle
                      + str(lotto_df['n2'][i])
                      + middle
                      + str(lotto_df['n3'][i])
                      + middle
                      + str(lotto_df['n4'][i])
                      + middle
                      + str(lotto_df['n5'][i])
                      + middle
                      + str(lotto_df['n6'][i])
                      + middle
                      + "now()"
                      + last
                     )
        print(insert_sql)
        zeroDb.update(insert_sql)
    
    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    zeroDb.update(insert_sql)
    zeroDb.commit()
    zeroDb.closedb()

def update_new_number():
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    
    update_sql1 = (
                    """
                    delete from temp; 
                    """
                  )
    update_sql2 = (
                    """
                    insert into temp 
                    select seq, n1, n2, n3, n4, n5, n6
                      from lotto
                     order by 1 desc;
                    """
                  )
    update_sql3 = (
                    """
                    drop table temp2; 
                    """
                  )
    update_sql4 = (
                    """
                    create table temp2
                    select seq
                         , no
                         , num
                      from (
                           select seq   
                                , 1  as no 
                                , n1 as num 
                             from temp 
                            union all
                            select seq   
                                 , 2  as no 
                                 , n2 as num 
                              from temp 
                             union all
                            select seq   
                                 , 3  as no 
                                 , n3 as num 
                              from temp 
                             union all
                            select seq   
                                 , 4  as no 
                                 , n4 as num 
                              from temp 
                             union all
                            select seq   
                                 , 5  as no 
                                 , n5 as num 
                              from temp 
                             union all
                             select seq   
                                 , 6  as no 
                                 , n6 as num 
                              from temp 
                           ) a
                    order by 1 desc, 2;
                    """
                  )
    update_sql5 = (
                    """
                    delete from temp3; 
                    """
                  )
    update_sql6 = (
                    """
                    insert into temp3
                    select a.*
                         , concat('#',seq,',',n1,',',n2,',',n3,',',n4,',',n5,',',n6) as cm
                         , (if(n1 between 1 and 9,1,0)   + if(n2 between 1 and 9,1,0)   + if(n3 between 1 and 9,1,0)   + if(n4 between 1 and 9,1,0)   + if(n5 between 1 and 9,1,0)   + if(n6 between 1 and 9,1,0))   as r0
                         , (if(n1 between 10 and 19,1,0) + if(n2 between 10 and 19,1,0) + if(n3 between 10 and 19,1,0) + if(n4 between 10 and 19,1,0) + if(n5 between 10 and 19,1,0) + if(n6 between 10 and 19,1,0)) as r1
                         , (if(n1 between 20 and 29,1,0) + if(n2 between 20 and 29,1,0) + if(n3 between 20 and 29,1,0) + if(n4 between 20 and 29,1,0) + if(n5 between 20 and 29,1,0) + if(n6 between 20 and 29,1,0)) as r2
                         , (if(n1 between 30 and 39,1,0) + if(n2 between 30 and 39,1,0) + if(n3 between 30 and 39,1,0) + if(n4 between 30 and 39,1,0) + if(n5 between 30 and 39,1,0) + if(n6 between 30 and 39,1,0)) as r3
                         , (if(n1 between 40 and 45,1,0) + if(n2 between 40 and 45,1,0) + if(n3 between 40 and 45,1,0) + if(n4 between 40 and 45,1,0) + if(n5 between 40 and 45,1,0) + if(n6 between 40 and 45,1,0)) as r4
                      from temp a; 
                    """
                  )              
    #print(update_sql)
    zeroDb.update(update_sql1)
    zeroDb.update(update_sql2)
    zeroDb.update(update_sql3)
    zeroDb.update(update_sql4)
    zeroDb.update(update_sql5)
    zeroDb.update(update_sql6)
    
    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    zeroDb.commit()
    zeroDb.closedb()

In [4]:
# 마지막 회차 확인 주소
main_url = "https://www.dhlottery.co.kr/gameResult.do?method=byWin"

# 마지막 회차 확인 함수
def GetLast():
    html = requests.get(main_url)
    soup = BeautifulSoup(html.text, "lxml")
    
    # meta 태그에서 id 속성이 desc이고 name 속성이 description인 
    # 태그를 찾아 content를 string으로 result에 저장
    result = str(soup.find("meta", {"id" : "desc", "name" : "description"})['content'])
    
    # 1. 회차 앞에 공란이 있고
    s_idx = result.find(" ") 
    
    # 2. 회차 다음에 '회'가 나오기 때문에 
    e_idx = result.find("회") 
    
    # 3. 공란 다음부터 '회' 이전까지가 마지막 회차이며 이것을 integer로 변환하여 리턴
    return int(result[s_idx + 1 : e_idx])

def GetStart():
     # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    print("접속완료")

    sql = ("""
           select max(seq) as seq
             from lotto
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    source = zeroDb.select(sql) 
    zeroDb.closedb()
    
    index = source[0]
    start = str(index)
    start2 = start.split(':')
    start3 = str(start2[1])
    start4 = start3.replace('}','')
    return int(start4)

In [5]:
if __name__ == '__main__':
    
    # 로또 회차정보 불러오기 
    startRound = GetStart()
    endRound   = GetLast()
    lotto_df   = getLottoWinInfo(startRound, endRound)
    insert_new_number(lotto_df)
    update_new_number()

접속완료
replace into zero.lotto values(1063,3,6,22,23,24,38,now());
replace into zero.lotto values(1064,3,6,9,18,22,35,now());


dsds